<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Four: Extension Activities</h2>	


TensorFlow (Optional)- Now, try using TensorFlow to categorize your images. The accuracy should be significantly higher due to the usage of nueral nets



In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

Using TensorFlow backend.


In [3]:
train_df = pd.read_pickle("starting_data.pkl").\
drop(labels=["filename", "aspect"], axis = 1)

In [4]:
#Normalizes images that are not 3-channel by setting all 3 channels equal
#to the first channel. 
#Input: Image (2 or 3 channel)
#Output: Image (3 channel normalized)
def normalize(image):
    if len(image.shape) == 2:
        temp = np.zeros((image.shape[0], image.shape[1], 3))
        for i in range(3):
            temp[:,:,i] = image
        image = temp
    return image

train_df['image'] = train_df['image'].apply(normalize)

In [5]:
#Transforms numpy array of images to ensure compatibility with TensorFlow
def transform(arr):
    new_arr = np.zeros((len(arr), 128, 128, 3))
    for i in range(len(arr)):
        new_arr[i] = arr[i]
    return new_arr

In [7]:
#Split the training dataframe into a training and validation set, then
#separate each set into features and labels. 

train, valid = train_test_split(train_df, test_size=0.1, random_state=42,\
                                shuffle=True, stratify=train_df['class'])

train_x, train_y = np.array(train['image']), np.array(train['class'])
train_x = transform(train_x / 255)
valid_x, valid_y = np.array(valid['image']), np.array(valid['class'])
valid_x = transform(valid_x / 255)

In [9]:
#Define a simple CNN model using Keras layers to perform deep learning. 
#The model consists of convolutional layers, pooling layers and some
#dense layers at the end with an output to a final layer of 20 nodes
#that will help us classify our images. Regularization is achieved
#with keras regularizers (specifically L2 loss) to prevent overfitting. 
#That said, with a small dataset like this it is very difficult to prevent
#overfitting. 
model = keras.Sequential()
model.add(Conv2D(32, (3,3), input_shape = (128, 128, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu',\
                kernel_regularizer=keras.regularizers.l2(0.001)))
model.add(Dense(64, activation = 'relu',\
                kernel_regularizer=keras.regularizers.l2(0.05)))
model.add(Dense(20, activation = 'sigmoid',\
                kernel_regularizer=keras.regularizers.l2(0.05)))

model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_x, train_y, batch_size = 128, epochs = 5)

Epoch 1/5
1350/1350 [==============================] - 51s 38ms/step - loss: 8.6752 - acc: 0.1089
Epoch 2/5
1350/1350 [==============================] - 50s 37ms/step - loss: 7.6691 - acc: 0.1207
Epoch 3/5
1350/1350 [==============================] - 53s 39ms/step - loss: 6.8710 - acc: 0.1222
Epoch 4/5
1350/1350 [==============================] - 55s 41ms/step - loss: 6.1662 - acc: 0.1089
Epoch 5/5
1350/1350 [==============================] - 58s 43ms/step - loss: 5.5403 - acc: 0.1244


In [11]:
#Evaluates the model on the unseen validation set
valid_loss, valid_acc = model.evaluate(valid_x, valid_y)
print('Validation loss is ', valid_loss)
print('Validation Accuracy is ', valid_acc)


151/151 [==============================] - 2s 14ms/step
Validation loss is  5.41227209331
Validation Accuracy is  0.152317878821
